# Title


Text

## Initialization
This section initializes the notebook.

### Dependencies
Here, all necessary libraries are imported.

In [1]:
#Add all dependencies to PYTHON_PATH
import sys
sys.path.append("/usr/lib/spark/python")
sys.path.append("/usr/lib/spark/python/lib/py4j-0.10.4-src.zip")
sys.path.append("/usr/lib/python3/dist-packages")
sys.path.append("/data/local/jupyterhub/modules/python")

#Define environment variables
import os
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "ipython"

import subprocess

#Load PySpark to connect to a Spark cluster
from pyspark import SparkConf, SparkContext
from hdfs import InsecureClient
from tempfile import TemporaryFile

#from osgeo import gdal
#To read GeoTiffs as a ByteArray
from io import BytesIO
from rasterio.io import MemoryFile

import numpy
import numpy as np
import pandas
import datetime
import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from os import listdir
from os.path import isfile, join
import scipy.linalg

### Configuration
This configuration determines whether functions print logs during the execution.

In [2]:
debugMode = True

### Connect to Spark
Here, the Spark context is loaded, which allows for a connection to HDFS.

In [3]:
appName = "plot_GeoTiff"
masterURL = "spark://emma0.emma.nlesc.nl:7077"

#A context needs to be created if it does not already exist
try:
    sc.stop()
except NameError:
    print("A new Spark Context will be created.")

sc = SparkContext(conf = SparkConf().setAppName(appName).setMaster(masterURL))
conf = sc.getConf()

A new Spark Context will be created.


## Subtitle

In [4]:
def getModeAsArray(filePath):
    data = sc.binaryFiles(filePath).take(1)
    byteArray = bytearray(data[0][1])
    memfile = MemoryFile(byteArray)
    dataset = memfile.open()
    array = np.array(dataset.read()[0], dtype=np.float64)
    memfile.close()
    array = array.flatten()
    array = array[~np.isnan(array)]
    return array

In [5]:
def detemineNorm(array1, array2):
    if array1.shape != array2.shape:
        print("Error: shapes are not the same: (" + str(array1.shape) + " vs " + str(array2.shape) + ")")
        return 0
    value = scipy.linalg.norm(array1 - array2)
    if value > 1:
        value = scipy.linalg.norm(array1 + array2)
    return value

In [6]:
textFile1 = sc.textFile("hdfs:///user/emma/svd/spark/BloomGridmetLeafGridmetCali3/U.csv").map(lambda line: (line.split(','))).map(lambda m: [ float(i) for i in m]).collect()

In [7]:
array1 = numpy.array(textFile1, dtype=np.float64)
vector11 = array1.T[0]
vector12 = array1.T[1]
vector13 = array1.T[2]

In [8]:
textFile2 = sc.textFile("hdfs:///user/emma/svd/BloomGridmetLeafGridmetCali/U.csv").map(lambda line: (line.split(','))).map(lambda m: [ np.float64(i) for i in m]).collect()

In [9]:
array2 = numpy.array(textFile2, dtype=np.float64).reshape(37,23926)
vector21 = array2[0]
vector22 = array2[1]
vector23 = array2[2]

In [11]:
array2.shape

(37, 23926)

In [15]:
print(detemineNorm(vector11, vector21))
print(detemineNorm(vector12, vector22))
print(detemineNorm(vector13, vector23))

0.022260136362918532
0.18853301319992985
0.5550399497443205


In [21]:
array1 = getModeAsArray("hdfs:///user/emma/svd/spark/BloomGridmetLeafGridmetCali3/u_tiffs/svd_u_0_26.tif")
array2 = getModeAsArray("hdfs:///user/emma/svd/BloomGridmetLeafGridmetCali/ModeU01.tif")
detemineNorm(array1, array2)

0.022260136362918532

In [22]:
print(detemineNorm(array1, vector11))
print(detemineNorm(array1, vector21))
print(detemineNorm(array2, vector11))
print(detemineNorm(array2, vector21))

0.0
0.022260136362918532
0.022260136362918532
0.0


In [23]:
~np.in1d(array1, vector21)

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [43]:
for i in range(10):
    print("%.19f %0.19f %0.19f" % (array1[i], array2[i], (array1[i]+array2[i])))

0.0023606640691764399 -0.0016509437782497344 0.0007097202909267055
0.0045640005668581788 -0.0020441078887899686 0.0025198926780682102
0.0053553859440128261 -0.0018522054259890808 0.0035031805180237453
0.0061622488178624847 0.0001264991065791714 0.0062887479244416560
0.0099863460134408232 -0.0041538877538869496 0.0058324582595538736
0.0101254175903585739 -0.0065606070472422192 0.0035648105431163547
0.0112847165878050149 -0.0045302850958463684 0.0067544314919586465
0.0095248069344842229 -0.0036491920046933923 0.0058756149297908306
0.0111647508148495587 -0.0032978926204460662 0.0078668581944034924
0.0121092885978979267 -0.0049129764878176564 0.0071963121100802703


### BloomFinalLowPR and LeafFinalLowPR

In [34]:
array1 = getModeAsArray("hdfs:///user/emma/svd/BloomFinalLowPRLeafFinalLowPR/ModeU01.tif")
array2 = getModeAsArray("hdfs:///user/emma/svd/spark/BloomFinalLowPRLeafFinalLowPR3/u_tiffs/svd_u_0_3.tif")
detemineNorm(array1, array2)

9.28098946087584e-16

In [35]:
array1 = getModeAsArray("hdfs:///user/emma/svd/BloomFinalLowPRLeafFinalLowPR/ModeU02.tif")
array2 = getModeAsArray("hdfs:///user/emma/svd/spark/BloomFinalLowPRLeafFinalLowPR3/u_tiffs/svd_u_1_3.tif")
detemineNorm(array1, array2)

8.816536190170412e-13

In [36]:
array1 = getModeAsArray("hdfs:///user/emma/svd/BloomFinalLowPRLeafFinalLowPR/ModeU01.tif")
array2 = getModeAsArray("hdfs:///user/emma/svd/spark/BloomFinalLowPRLeafFinalLowPR3/u_tiffs/svd_u_0_3.tif")
detemineNorm(array1, array2)

9.28098946087584e-16

### BloomGridmet and LeafGridmet

In [39]:
array1 = getModeAsArray("hdfs:///user/emma/svd/BloomGridmetLeafGridmetCali/ModeU01.tif")
array2 = getModeAsArray("hdfs:///user/emma/svd/spark/BloomGridmetLeafGridmetCali3/u_tiffs/svd_u_0_26.tif")
detemineNorm(array1, array2)

0.022260136362918532

In [40]:
array1 = getModeAsArray("hdfs:///user/emma/svd/BloomGridmetLeafGridmetCali/ModeU02.tif")
array2 = getModeAsArray("hdfs:///user/emma/svd/spark/BloomGridmetLeafGridmetCali3/u_tiffs/svd_u_1_26.tif")
detemineNorm(array1, array2)

0.18853301319992985

In [42]:
array1 = getModeAsArray("hdfs:///user/emma/svd/BloomGridmetLeafGridmetCali/ModeU03.tif")
array2 = getModeAsArray("hdfs:///user/emma/svd/spark/BloomGridmetLeafGridmetCali3/u_tiffs/svd_u_2_26.tif")
detemineNorm(array1, array2)

0.5550399497443205

End of Notebook